<h3>Code<h3

Dưới đây chúng ta sẽ lấy một ví dụ về cách sử dụng biến đổi khoảng cách cùng với đầu nguồn để phân đoạn các đối tượng chạm vào lẫn nhau. Hãy xem xét hình ảnh đồng xu bên dưới, các đồng tiền đang chạm vào nhau. Ngay cả khi bạn ngưỡng nó, nó sẽ chạm vào nhau.

![dongxu](dongxu.jpg)

Chúng tôi bắt đầu với việc tìm kiếm một ước tính gần đúng về các đồng tiền. Đối với điều đó, chúng ta có thể sử dụng nhị phân của OTSU.

In [2]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread('dongxu.jpg')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)

![dongxu1](dongxu1.jpg)

Bây giờ chúng ta cần loại bỏ tất cả nhiễu trắng nhỏ trong hình ảnh. Cho rằng chúng ta có thể sử dụng ___morphological opening___. Để loại bỏ bất kỳ lỗ nhỏ trong đối tượng, chúng ta có thể sử dụng ___morphological close___. Vì vậy, bây giờ chúng ta biết chắc chắn rằng khu vực gần trung tâm của các đối tượng là tiền cảnh và khu vực xa đối tượng là nền. Chỉ khu vực chúng tôi không chắc chắn là vùng biên của tiền xu. Vì vậy, chúng tôi cần trích xuất khu vực mà chúng tôi chắc chắn rằng chúng là tiền xu. Xói mòn loại bỏ các pixel ranh giới. Vì vậy, bất cứ điều gì còn lại, chúng tôi có thể chắc chắn đó là tiền xu. Điều đó sẽ hoạt động nếu các đối tượng không chạm vào nhau. Nhưng vì họ đang chạm vào nhau, một lựa chọn tốt khác sẽ là tìm biến đổi khoảng cách và áp dụng một ngưỡng thích hợp. Tiếp theo chúng ta cần tìm khu vực mà chúng tôi chắc chắn rằng chúng không phải là tiền xu. Đối với điều đó, chúng tôi giãn ra kết quả. Sự giãn nở làm tăng ranh giới đối tượng đến nền. Bằng cách này, chúng ta có thể đảm bảo bất kỳ khu vực nào trong kết quả trong kết quả thực sự là một nền tảng, vì vùng biên giới bị loại bỏ. Xem hình ảnh dưới đây.

___Opening___: Làm giảm nhiễu trắng

![opening](opening.PNG)

___Closing___: Làm giảm nhiễu đen

![closing](closing.PNG)

![region](region.jpg)

In [3]:
# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)

![distance](distance.jpg)

In [4]:
# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

In [5]:
markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]
cv.imshow("sure_bg",sure_bg)
cv.imshow("sure_fg",sure_fg)
# cv.imshow("thresh",thresh)
# cv.imshow("opening",opening)
cv.imshow("unknown",unknown)
cv.imshow("img",img)
cv.waitKey(0)

-1

![marker](marker.jpg)